Roc & Ethan 2023/2024 Project Decison in Uncertainty

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import product

1. Code a python class to represent a partially defined MDP.



In [ ]:
import numpy as np
import random

class PartiallyDefinedMDP:

    def __init__(self, states, actions, initial_state, discount_factor,
                 features, rewards ):
        self.states = states
        self.actions = actions
        self.initial_state = initial_state
        self.discount_factor = discount_factor
        self.features = features  # dictionnary where the key is the state and the value is a list of features {0,1} of size n the number of features
        self.rewards = rewards # list of rewards of size n the number of features


    def transition_function(self, s, action, s_prime):
        letter, number = s[0], int(s[1])
        letter_prime, number_prime = s_prime[0], int(s_prime[1])

        # if self.features[s][0] == 1 : # Il s'agit d'un etat but , sa valeur est fixée
        #     return 0

        #Cas où le mouvement est sur plus que une case, ou en diagonale
        if abs(ord(letter) - ord(letter_prime)) >1 :
            return 0 # La distance est supérieure à 1
        if abs(number - number_prime) > 1:
            return 0 # La distance est supérieure à 1
        if abs(ord(letter) - ord(letter_prime)) == 1 and abs(number - number_prime) == 1:
            return 0 # Déplacement en diagonale interdit


        # Cas ou il reste dans la même case
        #On recupere le dernier element de states (pour avoir la taille de la grille) on construit states de telle sorte que le dernier element soit la dernière case du tableau ie celle en haut a droite de l'exemple par exemple
        biggeststate = self.states[-1]
        biggestletter, biggestnumber = biggeststate[0], int(biggeststate[1])

        if s == s_prime:
            # Il reste dans la même case uniquement si l'action est dans une direction dans laquelle la grille est limitée
            # par exemple si on est en haut de la grille et qu'on veut aller en haut, on reste dans la même case
            if action == 'U' and ord(letter) == ord(biggestletter):
                return 0.8
            if action == 'D' and ord(letter) == ord('A'):
                return 0.8
            if action == 'L' and number == 1:
                return 0.8
            if action == 'R' and number == biggestnumber:
                return 0.8
            if (action =='U' or action=='D') and (number == 1 or number == biggestnumber) :
              return 0.1
            if (action =='L' or action=='R') and (ord(letter) == ord('A') or ord(letter) == ord(biggestletter)) :
              return 0.1

        # Cas ou s_prime est en haut ou en bas de s, alors que l'action est Droite ou Gauche
        if (action == 'R' or action == "L") and abs(ord(letter) - ord(letter_prime)) == 1  :
            return 0.1 # La probabilité de se déplacer en haut ou en bas avec une action
                       # d'aller à droite ou à gauche est de 0.1 dans le cas ou il slip vers le haut ou le bas


        # Cas ou s_prime est à gauche ou à droite de s, alors que l'action est haut ou bas
        if (action == 'U' or action == "D") and abs(number - number_prime) == 1  :
            return 0.1 # La probabilité de se déplacer à gauche ou à droite avec une action
                        # d'aller en haut ou en bas est de 0.1 dans le cas ou il slip vers la gauche ou la droite

        #Cas où tout se passe bien
        if action == 'U' and ord(letter) - ord(letter_prime) == -1 :
            return 0.8 # La probabilité de se déplacer en haut avec une action d'aller en haut est de 0.8
        if action == 'D' and ord(letter) - ord(letter_prime) == 1 :
            return 0.8 # La probabilité de se déplacer en bas avec une action d'aller en bas est de 0.8
        if action == 'R' and number - number_prime == -1 :
            return 0.8 # La probabilité de se déplacer à droite avec une action d'aller à droite est de 0.8
        if action == 'L' and number - number_prime == 1 :
            return 0.8 # La probabilité de se déplacer à gauche avec une action d'aller à gauche est de 0.8


        return 0 # Si aucune des conditions précédentes n'est vérifiée, alors la probabilité de se déplacer est de 0


    def reward_function(self, state):
        features = np.array(self.features[state])
        rewards = np.array(self.rewards)
        return np.dot(features, rewards)


2. Instantiate the gridworld running example.

In [ ]:
# Définir les états
states = []
for i in range(1, 9):
    for j in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        states.append(j+str(i))

# Définir les actions
actions = ['R', 'L', 'U', 'D']

# Groupes de features
treasures = ['A1']
bombs = ['C3', 'F2', 'F8', 'D8']
muds = ['B1', 'B2', 'B3', 'B4', 'C4', 'D3', 'D4', 'D5', 'D6']
waters = ['G2', 'G3']
mountains = ['C3']

discount_factor = 0.9

# Définir les features
features = {}
for state in states:
    feature_list = [0, 0, 0, 0, 0]  # Initialiser à [0, 0, 0, 0, 0]
    if state in treasures:
        feature_list[0] = 1  # Trésor
    if state in bombs:
        feature_list[1] = 1  # Bombe
    if state in muds:
        feature_list[2] = 1  # Boue
    if state in waters:
        feature_list[3] = 1  # Eau
    if state in mountains:
        feature_list[4] = 1  # Montagne
    features[state] = feature_list


# {Treasure: 0.04, Bomb: -0.05, Mud: -0.01, Water: -0.015, Mountain: -0.03}
rewards = [0.04,-0.05,-0.01,-0.015,-0.03]

# État initial
initial_state = 'D7'

# Instancier le GridWorld MDP
grid_world = PartiallyDefinedMDP(states, actions, initial_state, discount_factor, features, rewards)

3. Provide a method to create a similar but randomly generated gridworld MDP, where the size of the grid and the number of cells with each feature can be parameterized.

In [ ]:
import random
import string

class RandomGridWorld(PartiallyDefinedMDP):


    def __init__(self, grid_size, num_treasures, num_bombs, num_muds, num_waters, num_mountains,rewards):

        self.grid_size = grid_size
        self.states = []
        self.features = {
            'Treasure': [],
            'Bomb': [],
            'Mud': [],
            'Water': [],
            'Mountain': []
        }
        self.rewards = rewards

        # Générer les états
        letters = string.ascii_uppercase[:grid_size]
        for i in range(1, grid_size+1):
            for letter in letters:
                state = letter + str(i)
                self.states.append(state)

        # Assigner aléatoirement les features
        self.assign_features(num_treasures, num_bombs, num_muds, num_waters, num_mountains)

        # Définir les autres composants du MDP
        actions = [ 'R','L', 'U', 'D']
        initial_state = random.choice(self.states)
        discount_factor = 0.9
        super().__init__(self.states, actions, initial_state, discount_factor, self.features, self.rewards)

    def assign_features(self, num_treasures, num_bombs, num_muds, num_waters, num_mountains):
        states = self.states.copy()
        # Assigner les trésors
        self.features['Treasure'] = random.sample(states, num_treasures)
        # Assigner les bombes
        self.features['Bomb'] = random.sample(states, num_bombs)
        # Assigner les boues
        self.features['Mud'] = random.sample(states, num_muds)
        # Assigner les eaux
        self.features['Water'] = random.sample(states, num_waters)
        # Assigner les montagnes
        self.features['Mountain'] = random.sample(states, num_mountains)

        # Calculer les vecteurs de caractéristiques pour chaque état
        for state in self.states:
            # Créer un vecteur de caractéristiques pour chaque état, int(state in self.features[feature]) renvoie 1 si l'état est dans la liste des états de la feature,
            # 0 sinon
            feature_vector = [int(state in self.features[feature]) for feature in ['Treasure', 'Bomb', 'Mud', 'Water', 'Mountain']]
            self.features[state] = feature_vector

    def copy(self):
        return RandomGridWorld(self.grid_size, len(self.features['Treasure']), len(self.features['Bomb']), len(self.features['Mud']), len(self.features['Water']), len(self.features['Mountain']), self.rewards)


4. Code a policy iteration algorithm, to solve such an MDP if a reward function is provided.



In [ ]:
import numpy as np


class PolicyIteration:
    def __init__(self, mdp):
        self.mdp = mdp
        self.states = mdp.states
        self.actions = mdp.actions
        self.transition_function = mdp.transition_function
        self.reward_function = mdp.reward_function
        self.discount_factor = mdp.discount_factor
        # Initialiser une politique arbitraire, tous commencent avec la même action à droite
        self.policy = {s: 'R' for s in self.states}
        self.rewards = {s: self.mdp.reward_function(s) for s in self.states}
        self.values = [] # Correspond à la valeur de chaque état durant les itérations

    def evaluate_policy(self, policy):
        """ Évalue la valeur de chaque état pour la politique donnée """
        self.values = self.rewards.copy()  # Initialiser les valeurs à la reward de chaque état
        while True:
            values_changed = False
            for s in self.states:
                old_value = self.values[s]
                action = policy[s]
                new_value = self.get_quality_value(s,action) # On évalue la valeur de l'état s uniquement en fonction de la policy actuelle,
                                                                        #on ne prend pas en compte les autres actions
                if abs(old_value - new_value) > 1e-2:  # Seuil de convergence
                    values_changed = True
                self.values[s] = new_value
            if not values_changed: # SI aucune valeur n'a changé, on a convergé, on sort de la boucle
                break
        return self.values

    def extract_policy(self):
        """
        Extrait une nouvelle politique en maximisant la fonction de qualité # Policy Improvement Step
        π′(s) = argmaxa∈Aqπ(s, a)
        Teste toutes les actions possibles et retourne la valeur de la meilleure action
        -> retourne la politique optimale : π′(s) = argmaxa∈Aqπ(s, a) pour l'état s
        """
        new_policy = {}
        for s in self.states:
            new_policy[s] = max(self.actions, key=lambda a: self.get_quality_value(s, a))

        return new_policy

    def get_quality_value(self, state, action):
        value = self.reward_function(state) # R(s,a)
        for next_state in self.states:
            transition_prob = self.transition_function(state, action, next_state)
            value += self.discount_factor * transition_prob * self.values[next_state]
        return value

    def policy_iteration(self):
        iteration = 0
        while True:
            #print(f"Itération {iteration}:\n")
            values = self.evaluate_policy(self.policy)
            #print("Valeurs :")
            #for state, value in values.items():
               # print(f"État {state} : Valeur {value:.2f}")

            new_policy = self.extract_policy()
            #print("\nPolitique :\n")
            #for state, action in new_policy.items():
            #    print(f"État {state} : Action {action}")

            if new_policy == self.policy:
            #   print(f"\n--> Politique convergée avec {iteration} iterations.\n")
                break

            self.policy = new_policy
            iteration += 1

        return self.policy, values


Test de la classe Policy Iteration

In [ ]:
####################### EXAMPLE TD #######################


### Simple surcharge de la classe PartiallyDefinedMDP pour définir une fonction de transition
# spécifique à notre exemple avec les probabilité correcte pour l'état s2 (0.7 et 0.3)  ###

class PartiallyDefinedMDP_FOR_EXAMPLE(PartiallyDefinedMDP):
    def transition_function(self, s, action, s_prime):
        letter, number = s[0], int(s[1])
        letter_prime, number_prime = s_prime[0], int(s_prime[1])

        if self.features[s][0] == 1 : # Il s'agit d'un etat but , sa valeur est fixée
            return 0
        #Cas où le mouvement est sur plus que une case, ou en diagonale
        if abs(ord(letter) - ord(letter_prime)) >1 :
            return 0 # La distance est supérieure à 1
        if abs(number - number_prime) > 1:
            return 0 # La distance est supérieure à 1
        if abs(ord(letter) - ord(letter_prime)) == 1 and abs(number - number_prime) == 1:
            return 0 # Déplacement en diagonale interdit
        # Cas ou il reste dans la même case
        #On recupere le dernier element de states (pour avoir la taille de la grille) on construit states de telle sorte que le dernier element soit la dernière case du tableau ie celle en haut a droite de l'exemple par exemple
        biggeststate = self.states[-1]
        biggestletter, biggestnumber = biggeststate[0], int(biggeststate[1])
        if s == s_prime:
            if s == "A2": # unique
                return 0.3
            # Il reste dans la même case uniquement si l'action est dans une direction dans laquelle la grille est limitée
            # par exemple si on est en haut de la grille et qu'on veut aller en haut, on reste dans la même case
            if action == 'U' and ord(letter) == ord(biggestletter):
                if s == "A2":
                    return 0.7
                return 1
            if action == 'D' and ord(letter) == ord('A'):
                if s == "A2":
                    return 0.7
                return 1
            if action == 'L' and number == 1:
                if s == "A2":
                    return 0.7
                return 1
            if action == 'R' and number == biggestnumber:
                if s == "A2":
                    return 0.7
                return 1
            # Pour le reste des cas, il ne peut pas rester dans la même case... pas sur que ce soit utile, doit etre compris dans le return 0 de la fin
            else :
                return 0
        # Cas ou s_prime est en haut ou en bas de s, alors que l'action est Droite ou Gauche
        if (action == 'R' or action == "L") and abs(ord(letter) - ord(letter_prime)) == 1  :
            return 0 # La probabilité de se déplacer en haut ou en bas avec une action
                       # d'aller à droite ou à gauche est de 0.1 dans le cas ou il slip vers le haut ou le bas
        # Cas ou s_prime est à gauche ou à droite de s, alors que l'action est haut ou bas
        if (action == 'U' or action == "D") and abs(number - number_prime) == 1  :
            return 0 # La probabilité de se déplacer à gauche ou à droite avec une action
                        # d'aller en haut ou en bas est de 0.1 dans le cas ou il slip vers la gauche ou la droite
        #Cas où tout se passe bien
        if action == 'U' and ord(letter) - ord(letter_prime) == -1 :
            if s == "A2":
                return 0.7
            return 1 # La probabilité de se déplacer en haut avec une action d'aller en haut est de 0.8
        if action == 'D' and ord(letter) - ord(letter_prime) == 1 :
            if s == "A2":
                return 0.7
            return 1 # La probabilité de se déplacer en bas avec une action d'aller en bas est de 0.8
        if action == 'R' and number - number_prime == -1 :
            if s == "A2":
                return 0.7
            return 1 # La probabilité de se déplacer à droite avec une action d'aller à droite est de 0.8
        if action == 'L' and number - number_prime == 1 :
            if s == "A2":
                return 0.7
            return 1 # La probabilité de se déplacer à gauche avec une action d'aller à gauche est de 0.8
        return 0 # Si aucune des conditions précédentes n'est vérifiée, alors la probabilité de se déplacer est de 0

def print_grid(states, optimal_policy, optimal_values, features):
    grid_size = int(max(states, key=lambda s: int(s[1:]))[1:])
    grid = [['' for _ in range(grid_size)] for _ in range(len(set([s[0] for s in states])))]
    letters = [chr(ord('A') + i) for i in range(len(set([s[0] for s in states])))]
    letters.reverse()  # Inverser l'ordre des lettres

    # Calculer la largeur de l'espacement
    if grid_size < 5:
        spacing_width = 4
    else:
        spacing_width = 45 // grid_size

    # Print column numbers
    print(' ' * 3, end='')
    for col in range(1, grid_size + 1):
        print(f"{col:{spacing_width}}", end=' ')
    print()

    for row, letter in enumerate(letters):
        # Print row letter
        print(f"{letter:2}", end=' ')
        for col in range(grid_size):
            state = f"{letter}{col + 1}"
            if state in states:
                value = optimal_values[state]
                action = optimal_policy[state]
                treasure = features[state][0] == 1
                if treasure:
                    grid[row][col] = f"TTTT"
                else:
                    grid[row][col] = f"{value:.2f}"
                if action == 'R':
                    grid[row][col] += '→'
                elif action == 'L':
                    grid[row][col] += '←'
                elif action == 'U':
                    grid[row][col] += '↑'
                elif action == 'D':
                    grid[row][col] += '↓'
                # if treasure:
                #     grid[row][col] += 'T'
            print(f"{grid[row][col]:{spacing_width}}", end=' ')
        print()

states = ['A1', 'A2', 'A3', 'B1','B3', 'C1', 'C2', 'C3']

rewards = [2,-1 ] # 2 pour le trésor, -1 pour la bombe
# Treasure correspond à l'état but de notre exercice
actions = [ 'R','L', 'U', 'D']
initial_state = 'A1'
discount_factor = 0.9

# Groupes de features
treasures = ['C3']
bombs = ['C2']
features = {}
for state in states:
    feature_list = [0, 0]  # Initialiser à [0, 0, 0, 0, 0]
    if state in treasures:
        feature_list[0] = 1  # Trésor
    if state in bombs:
        feature_list[1] = 1  # Bombe

    features[state] = feature_list

mdp = PartiallyDefinedMDP_FOR_EXAMPLE(states, actions, initial_state, discount_factor, features, rewards)

print("\nQuel Exemple lancer ? :\n")
print("1 : Exemple du TD")
print("2 : Exemple Aléatoire")
boucle = 1

while boucle :
    try :
        choix = int(input("--> : "))
        boucle = False
    except :
        print("Veuillez entrer un chiffre")
        boucle = True

if choix == 1:
    # Créer une instance de PolicyIteration et résoudre le MDP
    policy_iteration = PolicyIteration(mdp)
    print("\nPolicy iteration pour exercice du TD :\n")
    optimal_policy, optimal_values = policy_iteration.policy_iteration()

    # Afficher la politique optimale
    print("\nPolitique optimale :")
    for state, action in optimal_policy.items():
        print(f"État {state} : Action {action}")

    # Afficher les valeurs optimales
    print("\nValeurs optimales :")
    for state, value in optimal_values.items():
        print(f"État {state} : Valeur {value:.2f}")

    print("\nGrille avec valeurs et policy optimales :\n")
    print_grid(mdp.states, optimal_policy, optimal_values, mdp.features)



# ############## EXAMPLE ALEATOIRE ####################

if choix == 2 :
    rewards = [ 2, -2, -0.5,-0.2,-1 ]

    # Définir une instance de MDP
    mdp = RandomGridWorld(9, 1, 1, 0, 0, 0,rewards)

    # Créer une instance de PolicyIteration et résoudre le MDP
    policy_iteration = PolicyIteration(mdp)
    print("\nPolicy iteration avec un exemple Aleatoire :\n")
    optimal_policy, optimal_values = policy_iteration.policy_iteration()

    # Afficher la politique optimale
    print("\nPolitique optimale :")
    for state, action in optimal_policy.items():
        print(f"État {state} : Action {action}")

    # Afficher les valeurs optimales
    print("\nValeurs optimales :")
    for state, value in optimal_values.items():
        print(f"État {state} : Valeur {value:.2f}")

    print("\nGrille avec valeurs et policy optimales :\n")
    print_grid(mdp.states, optimal_policy, optimal_values, mdp.features)


Quel Exemple lancer ? :

1 : Exemple du TD
2 : Exemple Aléatoire
--> : 1

Policy iteration pour exercice du TD :


Politique optimale :
État A1 : Action R
État A2 : Action R
État A3 : Action U
État B1 : Action D
État B3 : Action U
État C1 : Action D
État C2 : Action R
État C3 : Action R

Valeurs optimales :
État A1 : Valeur 1.26
État A2 : Valeur 1.40
État A3 : Valeur 1.62
État B1 : Valeur 1.13
État B3 : Valeur 1.80
État C1 : Valeur 1.02
État C2 : Valeur 0.80
État C3 : Valeur 2.00

Grille avec valeurs et policy optimales :

      1    2    3 
C  1.02↓ 0.80→ TTTT→ 
B  1.13↓      1.80↑ 
A  1.26→ 1.40→ 1.62↑ 


5. Provide a method which, given a reward function R, a number M of timesteps, and an initial state s0, provides a set O of M state-action pairs starting from s0. To simulate an imperfect tutor, the optimal action will be chosen with probability 0.95, and a random action is chosen otherwise.

In [ ]:
import random

def generate_stateActions(mdp, M, s0):
    """
    Le but de cette fonction est de générer des state-actions à partir d'un MDP en suivant la politique optimale.
    Le mdp pars de l'état initial s0 et génère M state-actions en suivant la politique optimale.
    Il se déplace de l'état courant à l'état suivant en suivant la politique optimale avec une probabilité de 0.95

    Args:
        reward_function (dict): Le dictionnaire des récompenses pour chaque caractéristique.
        M (int): Le nombre de state-actions à générer.
        s0 (str): L'état initial à partir duquel commencer les déplacements.

    Returns:
        list: Une liste de tuples (state, action) représentant les state-actions générées.
    """
    # Résoudre le MDP pour obtenir la politique optimale
    policy_iteration = PolicyIteration(mdp)
    optimal_policy, _ = policy_iteration.policy_iteration()

    # Initialiser l'état courant
    current_state = s0

    # Liste pour stocker les observations
    stateActions_pairs = []

    # Générer les stateActions_pairs
    for _ in range(M):

        # Avec une probabilité de 0.95, choisir l'action optimale
        # Sinon, choisir une action aléatoire
        if random.random() < 0.95:
            action = optimal_policy[current_state]
        else:
            action = random.choice(mdp.actions)

        # Ajouter la state_action à la liste
        stateActions_pairs.append((current_state, action))

        # Effectuer la transition vers le prochain état
        # Choisit un état au hasard de la liste mdp.states en utilisant les poids associés via la fonction de transition
        # le [0] à la fin car random.choices retourne une liste
        try :
            next_state = random.choices(mdp.states, weights=[mdp.transition_function(current_state, action, s_prime) for s_prime in mdp.states])[0]
        except :
            continue
        current_state = next_state

        # En gros, on récupère l'action optimale pour l'état courant à l'aide de la policy, puis, comme on a une part d'aléatoire
        # pour le déplacement, on choisit un état suivant en fonction de la probabilité de transition associée à chaque état en fonction
        # de l'action optimale choisie.

    return stateActions_pairs


# Générer 10 observations à partir de l'état initial 'A1'
stateActions = generate_stateActions(mdp, 10, 'A1')

for s_a in stateActions:
    print(s_a)

('A1', 'R')
('A2', 'R')
('A3', 'U')
('B3', 'U')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')


In [ ]:
#AFFICHAGE DES M STATES-ACTIONS calculées notamment grâce au mdp initialisé question 4

stateActions = generate_stateActions(mdp, 15, 'B3')

for s_a in stateActions:
    print(s_a)

('B3', 'U')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'D')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')
('C3', 'R')


6. Provide a method to compute the ratio P(R1|O)/P(R2|O) given R1, R2, O, and a choice for the
prior function. This is a meta step which will require to provide several classes and methods.

In [ ]:
import numpy as np
from abc import ABC, abstractmethod
from math import factorial


class Prior(ABC):
    """
    Classe abstraite pour représenter une distribution prior
    """

    @abstractmethod
    def compute_prior(self, R):
        """
        Calcule la probabilité a priori P(R) pour la fonction de récompense R.
        """
        pass

class UniformPrior(Prior):
    def __init__(self):
        pass

    def compute_prior(self, R):
        R_max = np.max(np.abs(R))
        # Vérifier si toutes les récompenses sont dans l'intervalle [-R_max, R_max]
        if np.all(np.abs(R) <= R_max):
            return 1.0 / (2 * R_max) ** len(R)
        else:
            return 0.0

class GaussianPrior(Prior):
    def __init__(self):
        pass

    def compute_prior(self, R):
        sigma = np.std(R) # Ecart type
        probs = [np.exp(-(r ** 2) / (2 * sigma ** 2)) / (sigma * np.sqrt(2 * np.pi)) for r in R] # On calcule la probabilité pour chaque feature r
        return np.prod(probs) # on fait le produit des probabilités pour chaque feature r

class BetaPrior(Prior):
    def __init__(self):
        pass

    def compute_prior(self, R):
        R_max = max(np.abs(R))
        alpha = len(R) + 1
        beta = len(R) + 1
        probs = []
        for r in R:
            if -R_max <= r <= R_max:
                x = (r + R_max) / (2 * R_max)  # Normaliser r dans [0, 1]
                prob = (gamma(alpha + beta) / (gamma(alpha) * gamma(beta))) * (x**(alpha - 1)) * ((1 - x)**(beta - 1))
                probs.append(prob)
            else:
                probs.append(0)  # Si r est hors de [-R_max, R_max], la probabilité est 0
        return np.prod(probs)


class MixturePrior(Prior):
    def __init__(self, priors, weights):
        self.priors = priors
        self.weights = weights

    def compute_prior(self, R):
        probs = [prior.compute_prior(R) * weight for prior, weight in zip(self.priors, self.weights)]
        return sum(probs)

class BayesianFramework:

    def __init__(self, mdp, prior, alpha):
        self.mdp = mdp
        self.prior = prior
        self.alpha = alpha

    def compute_likelihood(self, O, R):
        """
        Calcule la vraisemblance P(O|R) étant donné les observations O et la fonction de récompense R.
        """
        likelihood = 1.0
        # On remplace les rewards par R
        self.mdp.rewards = R
        policy_iteration = PolicyIteration(self.mdp)
        policy_iteration.policy_iteration() # On effectue la policy iteration pour obtenir les valeurs optimales de chaque état pour R
        values = policy_iteration.values
        # print("\nLe grid world avec les valeurs optimales pour R :",R, " est :\n")
        # print_grid(self.mdp.states, policy_iteration.policy, values, self.mdp.features)

        for s, a in O: # Uniquement les etats issus des observations
            value = self.mdp.reward_function(s) # Reward de l'état s
            # calcule la liste des quality pour chaque action possible pour l'état s
            q_values = [self.get_quality_value_R(s,a_prime,value,values) for a_prime in self.mdp.actions]
            #print("La quality values pour l'état ",s," est :\n\t ",q_values)
             # quality de l'action observée / somme des quality de toutes les actions possibles
            action_index = self.mdp.actions.index(a)
            #print("La quality values pour l'état ",s," et l'action ",a,"avec r :",R,  ", est :\n\t ",q_values[action_index])
            prob_optimal_action = np.exp(self.alpha * q_values[action_index]) / sum(np.exp(self.alpha * q_val) for q_val in q_values)
            likelihood *= prob_optimal_action

        return likelihood

    def compute_ratio(self, R1, R2, O):
        """
        Calcule le ratio P(R1|O) / P(R2|O) étant donné R1, R2 et les observations O.
        """
        likelihood_R1 = self.compute_likelihood(O, R1)
        likelihood_R2 = self.compute_likelihood(O, R2)
        prior_R1 = self.prior.compute_prior(R1)
        prior_R2 = self.prior.compute_prior(R2)

        ratio = (likelihood_R1 * prior_R1) / (likelihood_R2 * prior_R2)
        return ratio

    def get_quality_value_R(self, state, action,value,values):
        #print("get quality , State : ",state, "Action : ",action)

        for next_state in self.mdp.states:
            transition_prob = self.mdp.transition_function(state, action, next_state)
            #print("Transition prob : ",transition_prob," vers ",next_state)
            value += self.mdp.discount_factor * transition_prob * values[next_state]
            #print("Value : ",value)
        return value

# Test de la classe BayesianFramework


In [ ]:
rewards = [2,-2, -0.5,-0.2,-1]
R1 = [3,-1, -0.75,-0.3,-1.5]
R2 = [2,-2, -0.5,-0.2,-1]


mdp = RandomGridWorld(4, 1, 1, 0, 0, 0, rewards)
policy_iteration = PolicyIteration(mdp)
optimal_policy, optimal_values = policy_iteration.policy_iteration()
print_grid(mdp.states, optimal_policy, optimal_values, mdp.features)

# Créer des instances de distributions a priori
uniform_prior = UniformPrior()
gaussian_prior = GaussianPrior()
beta_prior = BetaPrior()
mixture_prior = MixturePrior([uniform_prior, gaussian_prior], [0.5, 0.5])

# Créer une instance de BayesianFramework
alpha = 1.0  # Paramètre de confiance dans le tuteur

# Générer 10 observations à partir de l'état initial 'A1'
observations = generate_stateActions(mdp, 10, 'A1')

# for s_a in stateActions:
#     print(s_a)

# Calculer le ratio de vraisemblance pour les fonctions de récompense R1 et R2
bayesian_framework = BayesianFramework(mdp, uniform_prior, alpha)
ratio = bayesian_framework.compute_ratio(R1, R2, observations)
print(f"Ratio de Likelihood Uniform Prior, P(R1|O) / P(R2|O) : {ratio}")

# Répéter avec d'autres distributions a priori
bayesian_framework = BayesianFramework(mdp, gaussian_prior, alpha)
ratio = bayesian_framework.compute_ratio(R1, R2, observations)
print(f"Ratio de Likelihood Gaussian Prior, P(R1|O) / P(R2|O) : {ratio}")

# bayesian_framework = BayesianFramework(mdp, beta_prior, alpha)
# ratio = bayesian_framework.compute_ratio(R1, R2, observations)
# print(f"Ratio de Likelihood Beta Prior, P(R1|O) / P(R2|O) : {ratio}")

bayesian_framework = BayesianFramework(mdp, mixture_prior, alpha)
ratio = bayesian_framework.compute_ratio(R1, R2, observations)
print(f"Ratio de Likelihood Mixture Prior, P(R1|O) / P(R2|O) : {ratio}")

      1    2    3    4 
D  10.85↓ 9.67← 8.61← 7.64← 
C  12.51↓ 10.99← 9.64← 8.45← 
B  TTTT← 12.34← 10.61← 9.15← 
A  10.27↑ 10.66↑ 9.48← 8.40← 
Ratio de Likelihood Uniform Prior, P(R1|O) / P(R2|O) : 1.136818489991992
Ratio de Likelihood Gaussian Prior, P(R1|O) / P(R2|O) : 3.817498482338822
Ratio de Likelihood Mixture Prior, P(R1|O) / P(R2|O) : 1.5448894508357254


7. Code the PolicyWalk algorithm to return a reward with high a posteriori probability.

In [ ]:
import numpy as np

def get_neighbors(R, delta, R_max):
    neighbors = []
    for i in range(len(R)):
        if -R_max <= R[i] + delta <= R_max:
            neighbor = R.copy()
            neighbor[i] += delta
            neighbors.append(neighbor)
        if -R_max <= R[i] - delta <= R_max:
            neighbor = R.copy()
            neighbor[i] -= delta
            neighbors.append(neighbor)
    return neighbors

def generate_reward_space(R_max, delta):
    """Génére l'espace des récompenses en utilisant une grille de taille delta."""
    R_space = []
    for i in range(int(2 * R_max /delta) + 1):
        R_space.append(-R_max + i * delta)
    return R_space

def MajDictio(tuple, dictioPosterioriProbas) :
    if tuple in dictioPosterioriProbas :
      dictioPosterioriProbas[tuple] += 1
    else :
      dictioPosterioriProbas[tuple] = 1

def quality_function(mdp, state, action, rewards, values):
    reward_vector = np.array(rewards)
    feature_vector = np.array(mdp.features[state])
    value = np.dot(feature_vector, reward_vector)
    for next_state in mdp.states:
        transition_prob = mdp.transition_function(state, action, next_state)
        value += mdp.discount_factor * transition_prob * values[next_state]
    return value

def policy_walk(prior, mdp, R_max, delta, num_iterations, alpha, observations):
    #Création d'un dictio où chaque clé est un tuple correspondant aux rewardlists et chaque valeur, le nombre de fois où ces rewards apparaissent
    dictioPosterioriProbas = {}
    R_space = generate_reward_space(R_max, delta) # On construit l'espace des rewards
    R = []
    for i in range(len(mdp.rewards)):
        R.append(random.choice(R_space)) # On initialise R avec des valeurs aléatoires dans l'espace des rewards
    MajDictio(tuple(R), dictioPosterioriProbas)
    mdpCopy = mdp.copy()
    mdpCopy.rewards = R
    policy_iteration = PolicyIteration(mdpCopy)
    optimal_policy, optimal_values = policy_iteration.policy_iteration()
    R_map = R.copy()  # Initialiser R_map avec les récompenses initiales

    for _ in range(num_iterations):
        neighbors = get_neighbors(R, delta, R_max)
        Rprime = neighbors[np.random.randint(len(neighbors))]

        accept = False
        for s in mdpCopy.states :
            val = quality_function(mdp, s, optimal_policy[s], Rprime, optimal_values)
            for a in mdpCopy.actions :
                if quality_function(mdp, s, a, Rprime, optimal_values) > val :
                    accept = True
                    break
            if accept == True :
                break

        BF = BayesianFramework(mdpCopy,prior, alpha)
        if accept == True :
            policy_iteration_bis = PolicyIteration(mdpCopy)
            policy_iteration_bis.policy = optimal_policy  #car on commence de la politique optimal_policy, on ne repart pas de 0
            optimal_policy_bis, optimal_values_bis = policy_iteration_bis.policy_iteration()


            if random.random() < min(1,BF.compute_ratio(Rprime, R, observations)) :
                R = Rprime
                optimal_policy, policyiter_values = optimal_policy_bis, optimal_values_bis

        elif random.random() < min(1,BF.compute_ratio(Rprime, R, observations)) :
            R = Rprime

        MajDictio(tuple(R), dictioPosterioriProbas)

    Rmap = list(max(dictioPosterioriProbas, key=dictioPosterioriProbas.get))

    return Rmap

# Test de la fonction PolicyWalk
# (Vous devez définir les classes manquantes comme RandomGridWorld, PolicyIteration, BayesianFramework, etc.)
rewards = [2, -2, -0.5, -0.5, -1]
mdp = RandomGridWorld(5, 1, 1, 0, 0, 0, rewards)
prior = UniformPrior()
stepsize = 0.5
nb_iterations = 10
R_max = max(np.abs(rewards))
alpha_bayesian_frame = 1.0
observation = generate_stateActions(mdp, 10, 'A1')
R_best = policy_walk(prior, mdp, R_max, stepsize, nb_iterations, alpha_bayesian_frame, observation)
print(f"Meilleure fonction de récompense R : {R_best}")

Meilleure fonction de récompense R : [1.0, 0.5, 0.5, 2.0, -0.5]


8. Provide such a modified version of your PolicyWalk algorithm.

9. Perform experiments on gridworld-like MDPs of varying characteristics. You will obtain a variety of plots giving the performance of your algorithms with different choices of parameters as a function of some other parameters (e.g., size of the MDP, number of features, number of iterations in the PolicyWalk algorithm, ...).

In [ ]:
import numpy as np

# Définir les paramètres à faire varier
grid_sizes = [3,4,5]
num_features = [[1, 1, 1, 1, 1]]
num_iterations = [3,5,10]
priors = [UniformPrior()]
alphas = [1]

# Dictionnaire pour stocker les résultats
results = {}

for grid_size in grid_sizes:
    for num_feat in num_features:
        for n_iter in num_iterations:
            for prior in priors:
                for alpha in alphas:

                    # Générer les gridworlds aléatoires et les observations
                    policy_losses = []
                    for _ in range(5):
                        mdp = RandomGridWorld(grid_size,num_feat[0],num_feat[1],num_feat[2],num_feat[3],num_feat[4] ,[2, -2, -0.5, -0.2, -1])
                        observations = generate_stateActions(mdp, 10, "A1")

                        # Exécuter PolicyWalk
                        R_map = policy_walk(prior, mdp, 1, 0.1, n_iter, alpha, observations)

                        # Évaluer la performance
                        mdp.rewards = R_map
                        policy_iteration = PolicyIteration(mdp)
                        optimal_policy, _ = policy_iteration.policy_iteration()

                        policy_loss = 0
                        for s, a in observations:
                            if a != optimal_policy[s]:
                                policy_loss += 1

                        policy_losses.append(policy_loss)

                    # Calculer les statistiques
                    mean_loss = np.mean(policy_losses)
                    std_loss = np.std(policy_losses)

                    # Stocker les résultats
                    key = (grid_size, tuple(num_feat), n_iter, str(prior), alpha)
                    results[key] = (mean_loss, std_loss)

# Afficher les résultats
for key, (mean_loss, std_loss) in results.items():
    grid_size, num_feat, n_iter, prior_str, alpha = key
    print(f"Grid Size: {grid_size}, #Features: {num_feat}, #Iterations: {n_iter}, Prior: {prior_str}, Alpha: {alpha}")
    print(f"Mean Policy Loss: {mean_loss:.3f}, Std Dev: {std_loss:.3f}")



import matplotlib.pyplot as plt

# Regrouper les résultats par (grid_size, num_feat)
grouped_results = {}
for key, (mean_loss, std_loss) in results.items():
    grid_size, num_feat, n_iter, prior_str, alpha = key
    group_key = (grid_size, tuple(num_feat))
    if group_key not in grouped_results:
        grouped_results[group_key] = []
    grouped_results[group_key].append((n_iter, mean_loss))

# Tracer les courbes
for group_key, group_data in grouped_results.items():
    grid_size, num_feat = group_key
    iterations = [x[0] for x in group_data]
    losses = [x[1] for x in group_data]

    plt.plot(iterations, losses, label=f"Grid Size: {grid_size}, #Features: {num_feat}")

plt.title("Policy Loss Curves")
plt.xlabel("Number of Iterations")
plt.ylabel("Policy Loss")
plt.legend()

# Afficher le graphique
plt.show()